# D1-D4 Multiple

In [2]:
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, Attention, Bidirectional, Concatenate, Conv1D,
                           Dense, Dropout, Embedding, Flatten, GlobalMaxPooling1D,
                           Input, Layer, LSTM, MaxPooling1D, Multiply, Permute,
                           RepeatVector, Reshape, SpatialDropout1D, TimeDistributed)
from keras.models import Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

import sys
sys.path.append('./cs16')
import cs16.prep as prep16
import cs16.plot as plot16
import cs16.build as build16
import cs16.DPF as DPF
imagesize = 64

import pandas as pd

#data_type = input("Enter '1. single' for single dataset or '2. multi' for multiple dataset: ")
data_type = '2'
if data_type == '1':
    file_path = 'single.txt'
    folder_path = './data/MVSA/single/'
elif data_type == '2':
    file_path = 'multi.txt'
    folder_path = './data/MVSA/multiple/'
else:
    print("Invalid input. Please enter either 'single' or 'multi'.")
    exit()

df = pd.read_csv(file_path, index_col=None, encoding='ISO-8859-1')

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
# Download stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def nlp_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regex
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into individual words
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words using Porter Stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

# Apply the preprocess_text function to the 'tweet' column of the dataframe
df['tweet'] = df['tweet'].apply(nlp_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
X_text, y_text = prep16.preprocess_text(df)
X_train_text, X_val_text, X_test_text, \
y_train_text, y_val_text, y_test_text = prep16.split_data(X_text, y_text, random_state=42)

X_polar, y_polar = prep16.preprocess_text(df,label = 'polarity')
X_train_polar, X_val_polar, X_test_polar, \
y_train_polar, y_val_polar, y_test_polar = prep16.split_data(X_polar, y_polar, random_state=42)

image_data_s, image_label_s = prep16.preprocess_images(df, folder_path, imagesize)
y_s = to_categorical(image_label_s, num_classes=3)

X_train_image, X_val_image, X_test_image, \
y_train_image, y_val_image, y_test_image= prep16.split_data(image_data_s, y_s, random_state=42)

y_train = to_categorical(y_train_polar, num_classes=3)
y_val =to_categorical(y_val_polar, num_classes=3)
y_test =to_categorical(y_test_polar, num_classes=3)
file_path

'multi.txt'

> # D1 - Late Fusion V08

In [7]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Softmax
from tensorflow.keras.models import Model
import numpy as np
start_time = time.time()
# ----- Text branch -----
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=100)(text_input)
t = LSTM(64, activation='tanh')(t)
text_branch = Dense(64, activation='tanh')(t)
text_out = Dense(3, activation='softmax', name='text_softmax')(text_branch)

# ----- Image branch -----
image_input = Input(shape=(64, 64, 3), name='image_input')
i = Conv2D(64, (3, 3), activation='relu', padding='same')(image_input)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = Conv2D(128, (3, 3), activation='relu', padding='same')(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)
i = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
i = Dropout(0.5)(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)
i = Flatten()(i)
image_branch = Dense(64, activation='relu')(i)
image_out = Dense(3, activation='softmax', name='image_softmax')(image_branch)

# ----- Define output model -----
model = Model(inputs=[text_input, image_input], outputs=[text_out, image_out])

# compile
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

# ---- Trainig----
history = model.fit(
    [X_train_text, X_train_image], 
    [y_train, y_train],       
    validation_data=([X_val_text, X_val_image], [y_val, y_val]),
    epochs=10,
    batch_size=64
)

# ----- Prediction & Late Fusion -----
# Seperated Probs
text_probs, image_probs = model.predict([X_test_text, X_test_image])
# Late Fusion：AVG
ensemble_probs = (text_probs + image_probs) / 2.0
predictions = np.argmax(ensemble_probs, axis=1)
true_labels = np.argmax(y_test, axis=1)
print(classification_report(true_labels, predictions, digits=4))

end_time = time.time()

# Time taken
elapsed_time = end_time - start_time

# print
print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")

Epoch 1/10
244/244 [==============================] - 96s 386ms/step - loss: 1.5219 - text_softmax_loss: 0.7123 - image_softmax_loss: 0.8096 - text_softmax_accuracy: 0.7474 - image_softmax_accuracy: 0.7294 - val_loss: 1.5270 - val_text_softmax_loss: 0.6263 - val_image_softmax_loss: 0.9007 - val_text_softmax_accuracy: 0.7786 - val_image_softmax_accuracy: 0.7227
Epoch 2/10
244/244 [==============================] - 115s 471ms/step - loss: 1.3239 - text_softmax_loss: 0.5848 - image_softmax_loss: 0.7391 - text_softmax_accuracy: 0.7727 - image_softmax_accuracy: 0.7470 - val_loss: 1.3499 - val_text_softmax_loss: 0.6470 - val_image_softmax_loss: 0.7028 - val_text_softmax_accuracy: 0.7766 - val_image_softmax_accuracy: 0.7704
Epoch 3/10
244/244 [==============================] - 128s 524ms/step - loss: 1.1957 - text_softmax_loss: 0.4646 - image_softmax_loss: 0.7311 - text_softmax_accuracy: 0.8162 - image_softmax_accuracy: 0.7458 - val_loss: 1.4388 - val_text_softmax_loss: 0.7331 - val_image_sof

> # D2 Multiple Early Fusion
(13/05/2025 test ok)

In [5]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np


# start
start_time = time.time()

# Text branch
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=100)(text_input)
t = LSTM(64, activation='tanh')(t)  # (batch, 64)

# Image branch
image_input = Input(shape=(64, 64, 3), name='image_input')
i = Conv2D(64, (3, 3), activation='relu', padding='same')(image_input)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = Conv2D(128, (3, 3), activation='relu', padding='same')(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)

# Layer2:
#i = Conv2D(32, (3, 3), activation=image_af, padding='same')(i)
#i = Dropout(0.5)(i)  # Add a dropout layer
#i = MaxPooling2D(pool_size=(2, 2))(i)
#i = BatchNormalization()(i)

i = Flatten()(i)  # (batch, N)


# Early fusion: 
fusion = Concatenate()([t, i])  # (batch, 64+N)


fusion = Dense(128, activation='relu')(fusion)
fusion = Dropout(0.5)(fusion)
fusion = Dense(64, activation='relu')(fusion)
output = Dense(3, activation='softmax')(fusion)

model = Model(inputs=[text_input, image_input], outputs=output)
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit([X_train_text, X_train_image], y_train, 
                    validation_data=([X_val_text, X_val_image], y_val),
                    epochs=10, 
                    batch_size=64)

# Evaluate the model
#test_loss, test_acc = model.evaluate([X_test_text, X_test_image], y_test)

from sklearn.metrics import classification_report
import numpy as np

predictions_prob = model.predict([X_test_text, X_test_image])

predictions = np.argmax(predictions_prob, axis=1)

true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))

#------------------------------------------------------------
output = some_function()
print(f"Function: {output}")
# End 
end_time = time.time()

#Compute time taken
elapsed_time = end_time - start_time

# Out
print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")

Epoch 1/10
244/244 [==============================] - 118s 475ms/step - loss: 0.8103 - accuracy: 0.7368 - val_loss: 0.6461 - val_accuracy: 0.7720
Epoch 2/10
244/244 [==============================] - 127s 519ms/step - loss: 0.6267 - accuracy: 0.7585 - val_loss: 0.6401 - val_accuracy: 0.7756
Epoch 3/10
244/244 [==============================] - 138s 565ms/step - loss: 0.5369 - accuracy: 0.7872 - val_loss: 0.6862 - val_accuracy: 0.7571
Epoch 4/10
244/244 [==============================] - 141s 577ms/step - loss: 0.4543 - accuracy: 0.8173 - val_loss: 0.7564 - val_accuracy: 0.7375
Epoch 5/10
244/244 [==============================] - 153s 626ms/step - loss: 0.3942 - accuracy: 0.8398 - val_loss: 0.8690 - val_accuracy: 0.7216
Epoch 6/10
244/244 [==============================] - 155s 635ms/step - loss: 0.3355 - accuracy: 0.8668 - val_loss: 1.0444 - val_accuracy: 0.6995
Epoch 7/10
244/244 [==============================] - 168s 690ms/step - loss: 0.2829 - accuracy: 0.8909 - val_loss: 1.1095 -

# MultiHeadAttention Transformer + EfficientNet 

# D3-Late Fusion

In [6]:
import numpy as np
import time
from tensorflow.keras.layers import (Input, Embedding, MultiHeadAttention, LayerNormalization,
                                     GlobalAveragePooling1D, Dense, GlobalAveragePooling2D, Concatenate)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
start_time = time.time()
#------------------------------------------------------------
# Text Transformer Encoder
#------------------------------------------------------------
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=128)(text_input)
attn = MultiHeadAttention(num_heads=4, key_dim=32)(t, t)
attn = LayerNormalization()(attn)
attn = GlobalAveragePooling1D()(attn)
text_feat = Dense(64, activation='relu')(attn)


text_prob = Dense(3, activation='softmax', name='text_softmax')(text_feat)

#------------------------------------------------------------
# Image Encoder using EfficientNetB0
#------------------------------------------------------------
image_input = Input(shape=(64, 64, 3), name='image_input')
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=image_input)
base_model.trainable = False
i = base_model.output
i = GlobalAveragePooling2D()(i)
image_feat = Dense(64, activation='relu')(i)


image_prob = Dense(3, activation='softmax', name='image_softmax')(image_feat)

#------------------------------------------------------------
# Model 
#------------------------------------------------------------
model = Model(inputs=[text_input, image_input], outputs=[text_prob, image_prob])

# loss/metric
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'text_softmax': 'categorical_crossentropy', 'image_softmax': 'categorical_crossentropy'},
    loss_weights={'text_softmax': 1.0, 'image_softmax': 1.0},
    metrics={'text_softmax': ['accuracy'], 'image_softmax': ['accuracy']}
)

#------------------------------------------------------------
# Training（兩個輸出都用同一份標籤；如需可分開）
#------------------------------------------------------------
history = model.fit(
    [X_train_text, X_train_image],
    [y_train, y_train],
    validation_data=([X_val_text, X_val_image], [y_val, y_val]),
    epochs=10,
    batch_size=64
)

#------------------------------------------------------------
# Evaluation — Late Fusion at inference
#------------------------------------------------------------
text_probs, image_probs = model.predict([X_test_text, X_test_image])

# Plan A：AVERAGE
ensemble_probs = 0.5 * (text_probs + image_probs)

# Plan B：
# import numpy as np
# logp = np.log(text_probs + 1e-12) + np.log(image_probs + 1e-12)
# ensemble_probs = np.exp(logp)
# ensemble_probs /= ensemble_probs.sum(axis=1, keepdims=True)

predictions = np.argmax(ensemble_probs, axis=1)
true_labels = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report
print("\nClassification Report (Late Fusion):")
print(classification_report(true_labels, predictions, digits=4))


end_time = time.time()


elapsed_time = end_time - start_time


print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")


Epoch 1/10
244/244 [==============================] - 104s 406ms/step - loss: 1.4873 - text_softmax_loss: 0.7394 - image_softmax_loss: 0.7479 - text_softmax_accuracy: 0.7436 - image_softmax_accuracy: 0.7440 - val_loss: 1.3603 - val_text_softmax_loss: 0.6575 - val_image_softmax_loss: 0.7028 - val_text_softmax_accuracy: 0.7704 - val_image_softmax_accuracy: 0.7704
Epoch 2/10
244/244 [==============================] - 113s 462ms/step - loss: 1.4108 - text_softmax_loss: 0.6654 - image_softmax_loss: 0.7454 - text_softmax_accuracy: 0.7560 - image_softmax_accuracy: 0.7469 - val_loss: 1.3326 - val_text_softmax_loss: 0.6236 - val_image_softmax_loss: 0.7089 - val_text_softmax_accuracy: 0.7833 - val_image_softmax_accuracy: 0.7704
Epoch 3/10
244/244 [==============================] - 123s 504ms/step - loss: 1.3388 - text_softmax_loss: 0.5924 - image_softmax_loss: 0.7465 - text_softmax_accuracy: 0.7722 - image_softmax_accuracy: 0.7469 - val_loss: 1.3580 - val_text_softmax_loss: 0.6601 - val_image_so

> ##  Early Fusion

In [8]:
from keras.layers import LSTM
from keras import regularizers
from keras.optimizers import Adam, Adagrad, RMSprop
from keras import backend as K
from keras import metrics
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, LayerNormalization, Concatenate
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

#------------------------------------------------------------
# Timer Start
#------------------------------------------------------------
start_time = time.time()

def some_function():
    result = 0
    for i in range(1000000):
        result += i
    return result

#------------------------------------------------------------
# Text Transformer Encoder
#------------------------------------------------------------
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=128)(text_input)
t = MultiHeadAttention(num_heads=4, key_dim=32)(t, t)
t = LayerNormalization()(t)
t = GlobalAveragePooling1D()(t)

#------------------------------------------------------------
# Image Encoder using EfficientNetB0
#------------------------------------------------------------
image_input = Input(shape=(64, 64, 3), name='image_input')
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=image_input)
base_model.trainable = False
i = base_model.output
i = GlobalAveragePooling2D()(i)

#------------------------------------------------------------
# Early Fusion
#------------------------------------------------------------
concatenated = Concatenate()([t, i])
fusion = Dense(64, activation='relu')(concatenated)
fusion = Dense(64, activation='relu')(fusion)
output = Dense(3, activation='softmax')(fusion)

model = Model(inputs=[text_input, image_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

#------------------------------------------------------------
# Training
#------------------------------------------------------------
history = model.fit([X_train_text, X_train_image], y_train, 
                    validation_data=([X_val_text, X_val_image], y_val),
                    epochs=10, 
                    batch_size=64)

#------------------------------------------------------------
# Evaluation
#------------------------------------------------------------
predictions_prob = model.predict([X_test_text, X_test_image])
predictions = np.argmax(predictions_prob, axis=1)
true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))
#------------------------------------------------------------
# Timer End
#------------------------------------------------------------
output = some_function()
print(f"\nOutput: {output}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"程式碼執行時間：{elapsed_time:.4f} 秒")


Epoch 1/10
244/244 [==============================] - 380s 2s/step - loss: 0.7479 - accuracy: 0.7469 - val_loss: 0.6965 - val_accuracy: 0.7704
Epoch 2/10
244/244 [==============================] - 366s 1s/step - loss: 0.6916 - accuracy: 0.7494 - val_loss: 0.6494 - val_accuracy: 0.7730
Epoch 3/10
244/244 [==============================] - 358s 1s/step - loss: 0.6179 - accuracy: 0.7635 - val_loss: 0.6290 - val_accuracy: 0.7802
Epoch 4/10
244/244 [==============================] - 367s 2s/step - loss: 0.5546 - accuracy: 0.7801 - val_loss: 0.6697 - val_accuracy: 0.7786
Epoch 5/10
244/244 [==============================] - 377s 2s/step - loss: 0.5073 - accuracy: 0.7957 - val_loss: 0.6990 - val_accuracy: 0.7571
Epoch 6/10
244/244 [==============================] - 365s 1s/step - loss: 0.4665 - accuracy: 0.8124 - val_loss: 0.7509 - val_accuracy: 0.7386
Epoch 7/10
244/244 [==============================] - 353s 1s/step - loss: 0.4387 - accuracy: 0.8264 - val_loss: 0.8365 - val_accuracy: 0.7036